In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import random

from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn.base import BaseEstimator, TransformerMixin

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_float32_matmul_precision('high')

import torch._dynamo
torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = True

import ray
from ray import tune, train
from ray.tune import CLIReporter, ExperimentAnalysis
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.train import Checkpoint


In [29]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.benchmark = False
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8'

In [3]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\DAILY\LUZ_Daily_Complete.csv'
data = pd.read_csv(input_file)
data = data.fillna(0)

In [4]:
def process_columns(df, value, substrings_ffill, substrings_interpolate):
    # Replace -999 with NaN
    df = df.replace(value, np.nan)
    
    # Forward fill for specified substrings
    ffill_cols = df.loc[:, df.columns.str.contains('|'.join(substrings_ffill), case=False)]
    ffill_cols = ffill_cols.ffill()
    
    # Interpolate for specified substrings
    interpolate_cols = df.loc[:, df.columns.str.contains('|'.join(substrings_interpolate), case=False)]
    interpolate_cols = interpolate_cols.interpolate(method='linear')
    
    return ffill_cols, interpolate_cols

columns_with_minus_999 = data.loc[:, (data == -999).any(axis=0)]

# Process columns
rainfall_cols, temp_cols_interpolated = process_columns(
    columns_with_minus_999, 
    -999, 
    substrings_ffill=['rainfall'], 
    substrings_interpolate=['tmax', 'tmin']
)
X = data.copy()
X[rainfall_cols.columns] = rainfall_cols
X[temp_cols_interpolated.columns] = temp_cols_interpolated
y = data[['GWAP','LWAP']]

In [5]:
train_size = int(0.6 * len(X))  # 60% for training
val_size = int(0.20 * len(X))   # 20% for validation
test_size = len(X) - train_size - val_size  # Remaining 15% for testing

train_data = X[:train_size]
train_labels = y[:train_size]

val_data = X[train_size:train_size + val_size]
val_labels = y[train_size:train_size + val_size]

test_data = X[train_size + val_size:]
test_labels = y[train_size + val_size:]
seq_len=7
batch_size=64

In [6]:
minmax_cols = []
boxcox_cols = []
yeojohnson_cols = []

# Classify columns into MinMax, BoxCox, or YeoJohnson families
def classify_features(data):
    for column in data.columns:
        col_data = data[column]
        skewness = col_data.skew()
        kurt = col_data.kurtosis()
        is_positive = np.all(col_data > 0)

        if -1 <= skewness <= 1 and -1 <= kurt <= 1:
            minmax_cols.append(column)  # MinMax family
        elif is_positive:
            boxcox_cols.append(column)  # BoxCox family
        else:
            yeojohnson_cols.append(column)  # YeoJohnson family

classify_features(X)

minmax_colsy = []
boxcox_colsy = []
yeojohnson_colsy = []

def classify_features(data):
    for column in data.columns:
        col_data = data[column]
        skewness = col_data.skew()
        kurt = col_data.kurtosis()
        is_positive = np.all(col_data > 0)

        if -1 <= skewness <= 1 and -1 <= kurt <= 1:
            minmax_colsy.append(column)  # MinMax family
        elif is_positive:
            boxcox_colsy.append(column)  # BoxCox family
        else:
            yeojohnson_colsy.append(column)  # YeoJohnson family

classify_features(y)


In [7]:
train_data_df = pd.DataFrame(train_data)  # Replace `data` with your actual data
val_data_df = pd.DataFrame(val_data)  # Replace `data` with your actual data
test_data_df = pd.DataFrame(test_data)  # Replace `data` with your actual data
# Test MinMaxScaler independently
minmax_test = MinMaxScaler(feature_range=(0, 1))
minmaxfit = minmax_test.fit(train_data_df[minmax_cols])
train_data_minmax = minmaxfit.transform(train_data_df[minmax_cols])
val_data_minmax = minmaxfit.transform(val_data_df[minmax_cols])
test_data_minmax = minmaxfit.transform(test_data_df[minmax_cols])

# Test Box-Cox + MinMaxScaler independently
boxcox_pipeline = Pipeline([
    ('boxcox', PowerTransformer(method='box-cox', standardize=False)),
    ('minmax', MinMaxScaler(feature_range=(0, 1)))
])
bc = boxcox_pipeline.fit(train_data_df[boxcox_cols])
train_data_bc = bc.transform(train_data_df[boxcox_cols])
val_data_bc = bc.transform(val_data_df[boxcox_cols])
test_data_bc = bc.transform(test_data_df[boxcox_cols])

# Test Yeo-Johnson + MinMaxScaler independently
yeojohnson_pipeline = Pipeline([
    ('yeojohnson', PowerTransformer(method='yeo-johnson', standardize=False)),
    ('minmax', MinMaxScaler(feature_range=(0, 1)))
])
yj = yeojohnson_pipeline.fit(train_data_df[yeojohnson_cols])
train_data_yj = yj.transform(train_data_df[yeojohnson_cols])
val_data_yj = yj.transform(val_data_df[yeojohnson_cols])
test_data_yj = yj.transform(test_data_df[yeojohnson_cols])

train_data_transformed = np.hstack([train_data_minmax, train_data_bc, train_data_yj])
val_data_transformed = np.hstack([val_data_minmax, val_data_bc, val_data_yj])
test_data_transformed = np.hstack([test_data_minmax, test_data_bc, test_data_yj])

In [8]:
train_labels_df = pd.DataFrame(train_labels)  # Replace `data` with your actual data
val_labels_df = pd.DataFrame(val_labels)  # Replace `data` with your actual data
test_labels_df = pd.DataFrame(test_labels)  # Replace `data` with your actual data
# Test MinMaxScaler independently


# Test Box-Cox + MinMaxScaler independently
boxcox_pipeline = Pipeline([
    ('boxcox', PowerTransformer(method='box-cox', standardize=False)),
    ('minmax', MinMaxScaler(feature_range=(0, 1)))
])
bcy = boxcox_pipeline.fit(train_data_df[boxcox_colsy])
train_labels_bc = bcy.transform(train_data_df[boxcox_colsy])
val_labels_bc = bcy.transform(val_data_df[boxcox_colsy])
test_labels_bc = bcy.transform(test_data_df[boxcox_colsy])

# Test Yeo-Johnson + MinMaxScaler independently
train_labels_transformed = np.hstack([train_labels_bc])
val_labels_transformed = np.hstack([ val_labels_bc])
test_labels_transformed = np.hstack([ test_labels_bc])

In [9]:
train_data_combined = np.hstack([train_data_minmax, train_data_bc, train_data_yj])
val_data_combined = np.hstack([val_data_minmax, val_data_bc, val_data_yj])
test_data_combined = np.hstack([test_data_minmax, test_data_bc, test_data_yj])

# Create DataFrames for each combined dataset
train_data_combined_df = pd.DataFrame(train_data_combined, columns=minmax_cols + boxcox_cols + yeojohnson_cols)
val_data_combined_df = pd.DataFrame(val_data_combined, columns=minmax_cols + boxcox_cols + yeojohnson_cols)
test_data_combined_df = pd.DataFrame(test_data_combined, columns=minmax_cols + boxcox_cols + yeojohnson_cols)

# Ensure all columns are displayed
pd.set_option('display.max_columns', None)

# Display the summary statistics for each column in the combined validation data
print("Summary statistics for each column in the train,val,test data:")
display(train_data_combined_df.describe(), val_data_combined_df.describe(), test_data_combined_df.describe())

Summary statistics for each column in the train,val,test data:


,FLOW_LUZ,Hourly Demand,TMAX_Cubi Point,TMAX_NAIA,TMIN_NAIA,TMAX_Science Garden,TMAX_San Jose,TMIN_San Jose,TMAX_Tayabas,TMIN_Tayabas,TMAX_CLSU,TMIN_CLSU,TMAX_Tanay,TMIN_Tanay,TMAX_Ambulong,TMAX_Casiguran,TMIN_Casiguran,TMAX_Clark,TMIN_Clark,TMAX_Calapan,TMIN_Calapan,GWAP,LWAP,TMIN_Cubi Point,TMIN_Science Garden,TMIN_Ambulong,RESERVE_GWAP_Fr,RESERVE_GWAP_Ru,RESERVE_GWAP_Rd,RESERVE_GWAP_Dr,RAINFALL_Cubi Point,RAINFALL_NAIA,RAINFALL_Science Garden,RAINFALL_San Jose,RAINFALL_Tayabas,RAINFALL_CLSU,RAINFALL_Tanay,RAINFALL_Ambulong,RAINFALL_Casiguran,RAINFALL_Clark,RAINFALL_Calapan
count,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000
mean,0.468299,0.648252,0.546937,0.590728,0.606996,0.565195,0.580500,0.536403,0.561562,0.552756,0.543888,0.598566,0.554953,0.666924,0.538093,0.555466,0.620166,0.609437,0.581394,0.611216,0.518960,0.606551,0.601150,0.492411,0.414022,0.446758,0.465425,0.516107,0.414435,0.377133,0.579450,0.456032,0.436821,0.494787,0.379439,0.568961,0.469711,0.527558,0.586225,0.491808,0.612988
std,0.203223,0.173484,0.169088,0.173105,0.166796,0.183004,0.169354,0.213272,0.191139,0.205733,0.167631,0.207601,0.205654,0.178767,0.186873,0.216838,0.163944,0.155433,0.180081,0.173074,0.178700,0.170671,0.170680,0.182261,0.176314,0.153151,0.228774,0.175465,0.104273,0.243130,0.206565,0.236578,0.269305,0.236016,0.244571,0.206949,0.241283,0.214956,0.215219,0.234339,0.189873
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.324398,0.544614,0.441441,0.477477,0.493827,0.434343,0.482143,0.361111,0.432432,0.428571,0.423554,0.472656,0.400000,0.549296,0.414414,0.392157,0.510204,0.503704,0.460843,0.517241,0.391304,0.489789,0.486298,0.388664,0.306930,0.353685,0.304974,0.409046,0.357732,0.198878,0.493149,0.386552,0.347275,0.417791,0.212722,0.493466,0.335037,0.413668,0.383035,0.422884,0.435920
50%,0.470000,0.662646,0.549550,0.594595,0.604938,0.575758,0.580357,0.472222,0.576577,0.607143,0.553719,0.625000,0.552381,0.718310,0.558559,0.588235,0.642857,0.614815,0.608434,0.632184,0.521739,0.584372,0.579811,0.495312,0.430661,0.454780,0.476836,0.498505,0.424483,0.405656,0.493149,0.386552,0.347275,0.417791,0.368485,0.493466,0.344686,0.413668,0.547165,0.422884,0.582796
75%,0.619960,0.777584,0.639640,0.700450,0.716049,0.696970,0.714286,0.750000,0.709459,0.696429,0.652893,0.765625,0.714286,0.788732,0.666667,0.725490,0.755102,0.718519,0.710843,0.747126,0.630435,0.746433,0.738104,0.618592,0.534411,0.539245,0.620082,0.640535,0.467362,0.568597,0.722833,0.619823,0.649551,0.637988,0.564978,0.691765,0.676540,0.697769,0.789462,0.621451,0.786879
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


,FLOW_LUZ,Hourly Demand,TMAX_Cubi Point,TMAX_NAIA,TMIN_NAIA,TMAX_Science Garden,TMAX_San Jose,TMIN_San Jose,TMAX_Tayabas,TMIN_Tayabas,TMAX_CLSU,TMIN_CLSU,TMAX_Tanay,TMIN_Tanay,TMAX_Ambulong,TMAX_Casiguran,TMIN_Casiguran,TMAX_Clark,TMIN_Clark,TMAX_Calapan,TMIN_Calapan,GWAP,LWAP,TMIN_Cubi Point,TMIN_Science Garden,TMIN_Ambulong,RESERVE_GWAP_Fr,RESERVE_GWAP_Ru,RESERVE_GWAP_Rd,RESERVE_GWAP_Dr,RAINFALL_Cubi Point,RAINFALL_NAIA,RAINFALL_Science Garden,RAINFALL_San Jose,RAINFALL_Tayabas,RAINFALL_CLSU,RAINFALL_Tanay,RAINFALL_Ambulong,RAINFALL_Casiguran,RAINFALL_Clark,RAINFALL_Calapan
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,0.462319,0.841316,0.600518,0.698815,0.725351,0.679120,0.575893,0.827182,0.735222,0.783268,0.683573,0.758455,0.709361,0.782124,0.683019,0.755909,0.784806,0.715373,0.706057,0.773658,0.866885,0.547668,0.547750,0.653095,0.564982,0.580869,0.306686,0.396148,-0.019063,0.158629,0.626125,0.501174,0.510468,0.534797,0.324867,0.629076,0.484759,0.549820,0.509254,0.542504,0.551526
std,0.185224,0.164575,0.191032,0.169632,0.165854,0.186608,0.180428,0.112834,0.136298,0.158137,0.165557,0.179922,0.183633,0.132843,0.167520,0.141552,0.115302,0.176049,0.148321,0.140960,0.188844,0.170823,0.174764,0.224561,0.221538,0.240512,0.238990,0.201831,0.138439,0.207967,0.236831,0.264479,0.271226,0.222858,0.190269,0.218323,0.245374,0.220948,0.205980,0.268478,0.199414
min,0.075273,0.286574,0.126126,0.144144,0.320988,0.040404,0.062500,0.444444,0.198198,0.339286,0.198347,0.062500,0.238095,0.422535,0.144144,0.431373,0.306122,0.133333,0.277108,0.195402,0.173913,0.204948,0.193783,0.168933,0.090310,0.018486,0.000000,0.014423,-0.062443,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.347785,0.749094,0.477477,0.594595,0.608025,0.595960,0.464286,0.768519,0.648649,0.696429,0.595041,0.687500,0.619048,0.718310,0.623874,0.666667,0.714286,0.666667,0.617470,0.701149,0.782609,0.420445,0.423292,0.533064,0.430661,0.470994,0.087748,0.238427,-0.062443,0.000022,0.493149,0.386552,0.347275,0.417791,0.212722,0.493466,0.335037,0.413668,0.383035,0.422884,0.435920
50%,0.434647,0.869941,0.594595,0.711712,0.740741,0.696970,0.584821,0.819444,0.756757,0.785714,0.719008,0.765625,0.742857,0.795775,0.707207,0.784314,0.795918,0.766667,0.710843,0.804598,0.891304,0.533827,0.542242,0.637676,0.534411,0.557184,0.301316,0.394328,-0.062443,0.000030,0.493149,0.386552,0.385298,0.417791,0.270214,0.493466,0.335037,0.413668,0.383035,0.422884,0.435920
75%,0.602658,0.974261,0.736486,0.835586,0.851852,0.785354,0.687500,0.879630,0.828829,0.875000,0.793388,0.859375,0.855952,0.873239,0.808559,0.882353,0.852041,0.837037,0.807229,0.862069,1.000000,0.660125,0.660893,0.760517,0.691822,0.667565,0.483357,0.527860,-0.062424,0.322743,0.866118,0.732528,0.760878,0.687006,0.440897,0.836375,0.726893,0.768000,0.684436,0.797450,0.725080
max,1.005873,1.095787,0.981982,1.045045,1.098765,1.020202,0.982143,1.092593,1.027027,1.321429,0.983471,1.109375,1.019048,1.056338,1.018018,1.127451,1.204082,1.007407,1.072289,1.045977,1.217391,0.872305,0.889567,1.874994,1.276220,1.493443,0.859863,0.796214,0.591384,0.732025,1.001455,0.958972,1.004991,0.994911,0.807591,0.991809,0.947618,0.920229,0.950460,0.993812,0.978044


,FLOW_LUZ,Hourly Demand,TMAX_Cubi Point,TMAX_NAIA,TMIN_NAIA,TMAX_Science Garden,TMAX_San Jose,TMIN_San Jose,TMAX_Tayabas,TMIN_Tayabas,TMAX_CLSU,TMIN_CLSU,TMAX_Tanay,TMIN_Tanay,TMAX_Ambulong,TMAX_Casiguran,TMIN_Casiguran,TMAX_Clark,TMIN_Clark,TMAX_Calapan,TMIN_Calapan,GWAP,LWAP,TMIN_Cubi Point,TMIN_Science Garden,TMIN_Ambulong,RESERVE_GWAP_Fr,RESERVE_GWAP_Ru,RESERVE_GWAP_Rd,RESERVE_GWAP_Dr,RAINFALL_Cubi Point,RAINFALL_NAIA,RAINFALL_Science Garden,RAINFALL_San Jose,RAINFALL_Tayabas,RAINFALL_CLSU,RAINFALL_Tanay,RAINFALL_Ambulong,RAINFALL_Casiguran,RAINFALL_Clark,RAINFALL_Calapan
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,0.557478,0.790429,0.538072,0.593546,0.685016,0.543171,0.560176,0.810185,0.658213,0.711840,0.639251,0.755886,0.590085,0.722458,0.563618,0.671501,0.736791,0.604718,0.642680,0.694221,0.822111,0.425797,0.413879,0.605789,0.490719,0.572939,0.248807,0.509619,0.020389,0.126388,0.584337,0.489098,0.484212,0.512489,0.409320,0.574505,0.525290,0.569485,0.595334,0.482560,0.599255
std,0.181921,0.150292,0.142749,0.129288,0.125611,0.164609,0.171825,0.092766,0.184081,0.135763,0.136037,0.111262,0.176289,0.113633,0.149362,0.188913,0.120253,0.104543,0.098211,0.172207,0.145781,0.129339,0.128884,0.141149,0.172058,0.159695,0.155736,0.155523,0.065825,0.162759,0.238283,0.249329,0.289652,0.260219,0.247500,0.192706,0.222899,0.170946,0.221186,0.259388,0.180647
min,0.030434,0.274650,0.099099,0.243243,0.283951,-0.010101,-0.071429,0.583333,0.198198,0.339286,0.190083,0.390625,0.190476,0.436620,0.180180,0.127451,0.459184,0.274074,0.349398,0.080460,0.391304,0.126463,0.117123,0.284784,0.143431,0.248106,0.000000,0.000000,-0.062443,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.413668,0.000000,0.000000,0.435920
25%,0.438704,0.702188,0.452703,0.513514,0.592593,0.444444,0.466518,0.743056,0.531532,0.642857,0.595041,0.687500,0.469048,0.647887,0.450450,0.553922,0.673469,0.548148,0.581325,0.563218,0.717391,0.331399,0.318027,0.495312,0.377928,0.470994,0.140552,0.449355,-0.051848,0.000022,0.493149,0.386552,0.347275,0.417791,0.212722,0.493466,0.335037,0.413668,0.383035,0.422884,0.435920
50%,0.558182,0.824704,0.576577,0.594595,0.679012,0.575758,0.553571,0.796296,0.666667,0.696429,0.661157,0.765625,0.600000,0.718310,0.576577,0.686275,0.734694,0.622222,0.650602,0.689655,0.826087,0.413897,0.399998,0.609240,0.473266,0.557184,0.248140,0.554680,0.021794,0.000058,0.493149,0.473708,0.465016,0.417791,0.403740,0.493466,0.512369,0.503364,0.575704,0.422884,0.514458
75%,0.693362,0.898801,0.648649,0.693694,0.787037,0.664141,0.707589,0.861111,0.790541,0.785714,0.727273,0.843750,0.721429,0.788732,0.682432,0.803922,0.816327,0.674074,0.710843,0.833333,0.913043,0.506575,0.495505,0.717940,0.583711,0.652418,0.357595,0.605170,0.065220,0.286576,0.760694,0.662005,0.710276,0.729482,0.632636,0.735342,0.717194,0.719330,0.812450,0.643084,0.769946
max,0.997687,1.025971,0.765766,0.855856,0.975309,0.888889,0.875000,1.074074,1.117117,1.107143,0.884298,1.000000,0.952381,1.253521,0.909910,1.029412,1.061224,0.785185,0.903614,1.034483,1.173913,0.871075,0.863151,0.973942,1.051195,1.198596,0.698703,0.853544,0.178230,0.614714,0.993971,0.937120,1.019252,0.988346,0.884560,0.953031,0.920012,0.926266,0.986848,0.987053,1.000699


In [10]:
class TimeSeriesDataset(Dataset):
    
    def __init__(self, X, y, seq_len):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.X) - self.seq_len 
        
    def __getitem__(self, idx):
        return (self.X[idx:idx+self.seq_len], self.y[idx+self.seq_len])

In [11]:
train_dataset = TimeSeriesDataset(train_data_transformed, train_labels_transformed, seq_len)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=False)

val_dataset = TimeSeriesDataset(val_data_transformed, val_labels_transformed, seq_len)    
val_dataloader = DataLoader(val_dataset, batch_size, shuffle=False) 

test_dataset = TimeSeriesDataset(test_data_transformed, test_labels_transformed, seq_len)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

In [12]:
class LSTMCustomCell(nn.Module):
    def __init__(self, input_size, hidden_size, activation_fn):
        super(LSTMCustomCell, self).__init__()
        self.hidden_size = hidden_size
        self.activation_fn = activation_fn
        
        # Combine all gate matrices into one large matrix for efficiency
        self.W_ih = nn.Linear(input_size, 4 * hidden_size, bias=False)
        self.W_hh = nn.Linear(hidden_size, 4 * hidden_size, bias=False)
        self.bias = nn.Parameter(torch.zeros(4 * hidden_size))
        
    def forward(self, x, hidden):
        h, c = hidden
        
        # Optimized matrix multiplication and bias addition
        gates = self.W_ih(x) + self.W_hh(h) + self.bias
        
        # Split into 4 gate vectors
        i_gate, f_gate, o_gate, g_gate = torch.chunk(gates, 4, dim=1)
        
        # Sigmoid activations for gates
        i_gate = torch.sigmoid(i_gate)
        f_gate = torch.sigmoid(f_gate)
        o_gate = torch.sigmoid(o_gate)
        
        # Apply the custom activation function for the cell gate
        g_gate = self.activation_fn(g_gate)
        
        # Compute the new cell state
        c_next = f_gate * c + i_gate * g_gate
        
        # Compute the new hidden state using the custom activation function
        h_next = o_gate * self.activation_fn(c_next)
        
        return h_next, c_next

In [13]:
class LSTMCustom(nn.Module):
    def __init__(self, input_size, hidden_size, dropout, num_layers=1, activation_fn=torch.tanh, batch_first=False):
        super(LSTMCustom, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.activation_fn = activation_fn
        self.batch_first = batch_first
        self.dropout = dropout
        # Create a list of LSTM cells
        self.cells = nn.ModuleList([LSTMCustomCell(input_size if i == 0 else hidden_size, hidden_size, activation_fn) for i in range(num_layers)])
        self.dropout_layer = nn.Dropout(dropout) if dropout > 0.0 else None
    def forward(self, x, hidden=None):
        # Determine the correct input shape
        if self.batch_first:
            batch_size, seq_len, _ = x.size()
            x = x.transpose(0, 1)  # Convert to (seq_len, batch_size, input_size) for processing
        else:
            seq_len, batch_size, _ = x.size()
        
        if hidden is None:
            # Initialize hidden and cell states with zeros
            h = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
            c = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
        else:
            h, c = hidden
        
        outputs = []
        
        # Iterate over each time step
        for t in range(seq_len):
            x_t = x[t, :, :]  # Input at time step t
            for i, cell in enumerate(self.cells):

                h[i], c[i] = cell(x_t, (h[i], c[i]))
                x_t = h[i]  # Pass hidden state to the next layer
                if self.dropout_layer and i < self.num_layers - 1:
                    x_t = self.dropout_layer(x_t)
            outputs.append(h[-1].unsqueeze(0))  # Collect output from the last layer
        
        # Stack the outputs across time steps
        outputs = torch.cat(outputs, dim=0)
        
        # Convert output back to (batch_size, seq_len, hidden_size) if batch_first is True
        if self.batch_first:
            outputs = outputs.transpose(0, 1)
        
        # Return outputs and the last hidden and cell states
        return outputs, (torch.stack(h), torch.stack(c))



In [14]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers,dropout,activation_fn):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = LSTMCustom(input_size, hidden_size, dropout, num_layers, activation_fn, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):

        # Forward propagate LSTM
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)  # Take the output from the last time step


        return out

In [234]:
#model = torch.compile(model)

In [15]:
input_size = train_data.shape[1]  # Number of features
hidden_size = 128
output_size = train_labels.shape[1]  # Number of output features
num_layers = 2
activation_fn = torch.relu
dropout=0.1
model = LSTMModel(input_size, hidden_size,output_size, num_layers,dropout,activation_fn).to(device)
epoch=100
# Define loss function and optimizer
criterion = nn.MSELoss()  # Use MSE loss for regression tasks
optimizer = optim.AdamW(list(model.parameters()), lr=1e-4, weight_decay=1e-5)


In [16]:
scaler = torch.amp.GradScaler()

def train(model, dataloader, device, optimizer, criterion):
    model.train()
    total_loss = 0.0  # Initialize total loss to 0

    
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Check for NaN values in inputs
        
        
        optimizer.zero_grad()
        outputs = model(inputs).to(device)
        
        # Check for NaN values in outputs
        
        
        loss = criterion(outputs, targets)
        
        
        
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        total_loss += loss.item() * inputs.size(0)

    # Return the average loss over all batches
    return total_loss / len(train_dataloader.dataset)
        
       

In [18]:
@torch.no_grad()
def evaluate(model, dataloader, device, criterion):
    model.eval()
    total_loss = 0.0  # Initialize total loss
    

    for i, (inputs, target) in enumerate(dataloader):  # Use `test_dataloader`
        inputs, target = inputs.to(device), target.to(device)

        # Forward pass
        outputs = model(inputs)
        
        predictions = outputs.to(device)  
    
        loss = criterion(predictions, target)

        # Accumulate the loss
        total_loss += loss.item() * inputs.size(0)

    # Return the average loss over all batches
    
    return total_loss/len(dataloader.dataset)


In [19]:
def load_data():
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=False)
    val_dataloader = DataLoader(val_dataset, batch_size, shuffle=False)
    return train_dataloader, val_dataloader

In [20]:
def train_lstm(config,epoch):
    set_seed(args.seed + i)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    activation_fn_mapping = {
        "tanh": torch.tanh,
        "relu": torch.relu,
    }
    activation_fn = activation_fn_mapping[config["activation_fn"]]
    model = LSTMModel(input_size, config["hidden_size"], output_size, num_layers,config["dropout"],activation_fn=activation_fn).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(list(model.parameters()), lr=config["lr"], weight_decay=config["weight_decay"])
    
    

    train_dataloader, val_dataloader = load_data()
    for e in range(epoch):  # Replace with your actual number of epochs
        train_loss = train(model, train_dataloader, device, optimizer, criterion)
        val_loss = evaluate(model, val_dataloader, device, criterion)
        

        # Report the results
        session.report(  # Highlighted change
            {"loss": val_loss} # Highlighted change
        )


In [21]:
search_space = {
    "hidden_size": tune.choice([32, 64, 128, 256]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "weight_decay": tune.loguniform(1e-6, 1e-3),
    "dropout": tune.uniform(0.1, 0.5),
    "activation_fn": tune.choice(["tanh", "relu"])
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    metric_columns=["loss", "training_iteration"]
)

def trial_dirname_creator(trial):
    return f"trial_{trial.trial_id}"

In [22]:
ray.shutdown()  # Shutdown any existing Ray instances
ray.init(ignore_reinit_error=True)

Usage stats collection is enabled. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2024-10-26 20:35:02,877	INFO worker.py:1807 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.20
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8265


In [23]:
bestresults = tune.run(
    tune.with_parameters(train_lstm, epoch=epoch),
    resources_per_trial={"cpu": 10, "gpu": 1},
    config=search_space,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter,
    verbose=1,
    trial_dirname_creator=trial_dirname_creator
)

2024-10-26 20:35:06,191	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-10-26 20:35:06 (running for 00:00:00.18)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 10.0/20 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: C:/Users/PAULOJ~1/AppData/Local/Temp/ray/session_2024-10-26_20-35-00_587879_29916/artifacts/2024-10-26_20-35-06/train_lstm_2024-10-26_20-35-06/driver_artifacts
Number of trials: 10/10 (10 PENDING)
+------------------------+----------+-------+-----------------+-----------+---------------+-------------+----------------+
| Trial name             | status   | loc   | activation_fn   |   dropout |   hidden_size |          lr |   weight_decay |
|------------------------+----------+-------+-----------------+-----------+---------------+-------------+----------------|
| train_lstm_baea6_00000 | PENDING  |       | tanh            |  0.371047 |           256 | 0.00329389  |    3.7761e-06  |
| train_lstm_baea6_00001 |

2024-10-26 20:35:51,646	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/Paulo John Mercado/ray_results/train_lstm_2024-10-26_20-35-06' in 0.0260s.
2024-10-26 20:35:51,659	INFO tune.py:1041 -- Total run time: 45.47 seconds (45.40 seconds for the tuning loop).


== Status ==
Current time: 2024-10-26 20:35:51 (running for 00:00:45.42)
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -0.03941889053626026 | Iter 4.000: -0.04523364341275083 | Iter 2.000: -0.08027059053533153 | Iter 1.000: -0.25802666171313193
Logical resource usage: 10.0/20 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: C:/Users/PAULOJ~1/AppData/Local/Temp/ray/session_2024-10-26_20-35-00_587879_29916/artifacts/2024-10-26_20-35-06/train_lstm_2024-10-26_20-35-06/driver_artifacts
Number of trials: 10/10 (10 TERMINATED)
+------------------------+------------+-----------------+-----------------+-----------+---------------+-------------+----------------+-----------+----------------------+
| Trial name             | status     | loc             | activation_fn   |   dropout |   hidden_size |          lr |   weight_decay |      loss |   training_iteration |
|------------------------+------------+-----------------+-----------------+-----------+---------------+-------

In [24]:
best_config = bestresults.get_best_config(metric="loss", mode="min")
print("Best hyperparameters found were: ", best_config)
best_trial = bestresults.get_best_trial(metric="loss", mode="min")
best_loss = best_trial.last_result["loss"]
print("Best loss found was: ", best_loss)

Best hyperparameters found were:  {'hidden_size': 32, 'lr': 0.0021977699527409513, 'weight_decay': 0.00014056950796939994, 'dropout': 0.48663414633672786, 'activation_fn': 'relu'}
Best loss found was:  0.024503708716562326


In [25]:
activation_fn_mapping = {
    "tanh": torch.tanh,
    "relu": torch.relu,
}
optimizedactivation_fn = activation_fn_mapping[best_config["activation_fn"]]
optimizedmodel=LSTMModel(input_size, best_config["hidden_size"], output_size, num_layers,best_config["dropout"],activation_fn=optimizedactivation_fn).to(device)

In [26]:
def run(optimizedmodel, train_dataloader, val_dataloader, test_dataloader, device, epoch,optimizer):
    train_losses = []
    val_losses = []
    test_losses = []
    for epoch in range(epoch):
        
        train_loss = train(optimizedmodel, train_dataloader, device, optimizer, criterion)
        train_losses.append(train_loss)
        val_loss=evaluate(optimizedmodel,val_dataloader,device,criterion)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}, Training Loss: {train_loss}, Val Loss: {val_loss}")
        
        if train_loss < 1e-3 :
            break
    last_train_loss = train_losses[-1]
    last_val_loss = val_losses[-1]
    return last_train_loss, last_val_loss

In [27]:
epoch=100
run(model, train_dataloader,val_dataloader, test_dataloader, device, epoch,optimizer)

Epoch 1, Training Loss: 0.3403483028685687, Val Loss: 0.27315151954083133
Epoch 2, Training Loss: 0.32575094703316965, Val Loss: 0.25770379270581034
Epoch 3, Training Loss: 0.3119842650027397, Val Loss: 0.24294601364744653
Epoch 4, Training Loss: 0.29741116186167416, Val Loss: 0.2274563419733116
Epoch 5, Training Loss: 0.2811389591743664, Val Loss: 0.20965452899606965
Epoch 6, Training Loss: 0.2624370340612261, Val Loss: 0.18674366401253845
Epoch 7, Training Loss: 0.23734276131521523, Val Loss: 0.15537187348542025
Epoch 8, Training Loss: 0.2016878852020007, Val Loss: 0.11141873226105738
Epoch 9, Training Loss: 0.14931791007795356, Val Loss: 0.05629867410434664
Epoch 10, Training Loss: 0.076731584499441, Val Loss: 0.05466417223215103
Epoch 11, Training Loss: 0.03611364306367744, Val Loss: 0.1972453366938255
Epoch 12, Training Loss: 0.05357942102680074, Val Loss: 0.10264475600753757
Epoch 13, Training Loss: 0.03761598168334247, Val Loss: 0.05791579690768564
Epoch 14, Training Loss: 0.037

(0.013866842073969664, 0.028705863000677644)

In [31]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Luzon GWAP: Training, Validation, and Test Losses Over Epochs')

plt.show()


NameError: name 'train_losses' is not defined

<Figure size 1000x500 with 0 Axes>